<div style="
    display: flex; 
    background-color: #4CAF50; 
    border-radius: 15px; 
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
  <h1 style="
      margin: auto; 
      padding: 40px; 
      color: #fff; 
      font-family: 'Arial', sans-serif; 
      text-align: center;">
    Projet: Détectez des faux billets avec R ou Python
  </h1>
</div>

<div style="
    background-color: #FF5722; 
    border-radius: 10px; 
    padding: 25px; 
    margin: 20px 0; 
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
  <h2 style="
      color: #fff; 
      font-family: 'Arial', sans-serif; 
      text-align: center; 
      margin: 0;">
    Étape 1 - Importation des librairies et chargement des fichiers
  </h2>
</div>

In [1]:
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

<div style="
    background-color: #FF5722; 
    border-radius: 10px; 
    padding: 25px; 
    margin: 20px 0; 
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
  <h2 style="
      color: #fff; 
      font-family: 'Arial', sans-serif; 
      text-align: center; 
      margin: 0;">
    Étape 2 - Fonction qui classifie les billets
  </h2>
</div>

In [2]:
def classifier_vrai_faux_billet(model, fichier_csv, threshold=None):
    
    # Charger les données
    df = pd.read_csv(fichier_csv)
    
    # Vérifier que le modèle a bien une méthode "predict"
    if not hasattr(model, "predict"):
        raise ValueError("Le modèle fourni n'est pas valide ou n'a pas été entraîné.")
        
    # Sélection des données significatives de la régression logistique
    X = df[['diagonal', 'height_left', 'height_right','margin_low','margin_up','length']]
   
    # Standardisation des données
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X.values)
    
    # Faire les prédictions
    predictions_proba = model.predict_proba(X_scaled)[:, 1]  # Probabilité pour la classe "True"
    
    # Si threshold est renseigné, on ajuste les prédictions en fonction de ce seuil
    if threshold is not None:
        y_pred = ["True" if x > threshold else "False" for x in predictions_proba]  # Applique le seuil
    else:
        # Sinon, utiliser les prédictions de base (classe majoritaire)
        predictions = model.predict(X_scaled)
        y_pred = ["True" if pred == 1 else "False" for pred in predictions]  # Conversion en True/False
    
    # Affichage des résultats
    df_pred = df.copy()
    df_pred['prediction'] = y_pred
    df_pred['probabilité vrai %'] = np.round(predictions_proba * 100, 2)
    
    return df_pred


<div style="
    background-color: #FF5722; 
    border-radius: 10px; 
    padding: 25px; 
    margin: 20px 0; 
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
  <h2 style="
      color: #fff; 
      font-family: 'Arial', sans-serif; 
      text-align: center; 
      margin: 0;">
    Étape 3 - Importation du modèle
  </h2>
</div>

In [3]:
# Charger le modèle
GaussianNB_model = joblib.load('GaussianNB_model.pkl')

<div style="
    background-color: #FF5722; 
    border-radius: 10px; 
    padding: 25px; 
    margin: 20px 0; 
    box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
  <h2 style="
      color: #fff; 
      font-family: 'Arial', sans-serif; 
      text-align: center; 
      margin: 0;">
    Étape 4 - Utilisation de la fonction "classifier_vrai_faux_billet"
  </h2>
</div>

In [4]:
classifier_vrai_faux_billet(GaussianNB_model, 'billets_test.csv', threshold=0.8)

,diagonal,height_left,height_right,margin_low,margin_up,length,id,prediction,probabilité vrai %
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1,True,100.00
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2,False,0.43
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3,True,100.00
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4,False,0.00
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5,False,8.43
